<a href="https://colab.research.google.com/github/43pratik/ML_lab/blob/main/Prac_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

In [2]:
file_path = "/content/House Price India.csv"
df = pd.read_csv(file_path)
print(df.head())
print(df.info())

           id   Date  number of bedrooms  number of bathrooms  living area  \
0  6762810635  42491                   4                 2.50         2920   
1  6762810998  42491                   5                 2.75         2910   
2  6762812605  42491                   4                 2.50         3310   
3  6762812919  42491                   3                 2.00         2710   
4  6762813105  42491                   3                 2.50         2600   

   lot area  number of floors  waterfront present  number of views  \
0      4000               1.5                   0                0   
1      9480               1.5                   0                0   
2     42998               2.0                   0                0   
3      4500               1.5                   0                0   
4      4750               1.0                   0                0   

   condition of the house  ...  Built Year  Renovation Year  Postal Code  \
0                       5  ...    

In [5]:
df["number of room"] = df["number of bedrooms"] + df["number of bathrooms"]
features = ["living area", "number of bedrooms", "number of bathrooms", "number of room"]
target = "SALES_PRICE"

In [6]:
df[features] = df[features].fillna(df[features].median())

In [9]:
print(df.columns)  # Print the available column names

Index(['id', 'Date', 'number of bedrooms', 'number of bathrooms',
       'living area', 'lot area', 'number of floors', 'waterfront present',
       'number of views', 'condition of the house', 'grade of the house',
       'Area of the house(excluding basement)', 'Area of the basement',
       'Built Year', 'Renovation Year', 'Postal Code', 'Lattitude',
       'Longitude', 'living_area_renov', 'lot_area_renov',
       'Number of schools nearby', 'Distance from the airport', 'Price',
       'number of room'],
      dtype='object')


In [10]:
target = "correct_column_name"  # Replace with the correct column name

In [11]:
target = "Sale Price"

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Load the dataset
file_path = "House_Price_Data.csv"  # Change this to your actual file path
df = pd.read_csv("/content/House Price India.csv")

# Create 'age of the house' feature
df["age of the house"] = 2025 - df["Built Year"]  # Assuming current year is 2025

# Select relevant features and target variable
features = ["living area", "N_ROOM", "age of the house"]
target = "Price"

X = df[features]
y = df[target]

# Split the data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Print evaluation metrics
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

KeyError: "['N_ROOM'] not in index"